<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/mediapipe_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe
!git clone "https://github.com/Kazuhito00/iris-detection-using-py-mediapipe.git"

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


     |████████████████████████████████| 37.7MB 60.4MB/s 
Cloning into 'iris-detection-using-py-mediapipe'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 15), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (42/42), done.
demo.py  face_mesh  iris_landmark  LICENSE  README.md  utils


In [2]:
import os

path = '/content/iris-detection-using-py-mediapipe'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

demo.py  face_mesh  iris_landmark  LICENSE  README.md  utils


In [3]:
!python3 demo.py

2021-06-08 06:52:15.583454: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[ WARN:0] global /io/opencv/modules/videoio/src/cap_v4l.cpp (802) open VIDEOIO ERROR: V4L: can't open camera by index 0
W20210608 06:52:17.106629   126 tflite_model_loader.cc:32] Trying to resolve path manually as GetResourceContents failed: ; Can't find file: mediapipe/modules/face_detection/face_detection_front.tflite
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W20210608 06:52:17.110426   127 tflite_model_loader.cc:32] Trying to resolve path manually as GetResourceContents failed: ; Can't find file: mediapipe/modules/face_landmark/face_landmark.tflite
